In [ ]:
import json
!pip install pulp
from pulp import *
import time
# --- INÍCIO DA MEDIÇÃO ---
start_time = time.time()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 87.8 MB/s eta 0:00:00


In [ ]:
#Input data
with open('/content/instance7.json') as f:
    instance = json.load(f)

#Sets and parameters
days = ["L", "Ma", "Mi", "J", "V"]
employee = instance['Employees']
desks = instance['Desks']
zones = instance['Zones']
desks_per_zone = instance['Desks_Z']
desks_per_employee = instance['Desks_E']
employee_per_group = instance['Employees_G']
preferable_days = instance['Days_E']
weight = 5
groups = [g for g in employee_per_group]

#Update weight matrix
weights = [[1 for _ in employee] for _ in days]
for e in employee:
    for i in range(len(preferable_days[e])):
        day_index = days.index(preferable_days[e][i])
        weights[day_index][employee.index(e)]=weight

#Identify the employees that can use each desk
employee_per_desk = {}
for desk_id in desks:
    employees_in_desk = []
    for e in employee:
        possible_desks = desks_per_employee[e]
        for m in possible_desks:
            if m == desk_id:
                employees_in_desk.append(e)
    employee_per_desk[desk_id]=employees_in_desk

In [ ]:
#Problem Initialization
assignment = LpProblem("StudentChallenge", LpMaximize)

#Decision Variables
x = LpVariable.dicts("x", [(d,e,k) for d in days for e in employee for k in desks if e in employee_per_desk[k]], cat = "Binary")
u = LpVariable.dicts("u", [(d,g) for d in days for g in groups], cat = "Binary")

#Objective Function
assignment += lpSum([weights[days.index(d)][employee.index(e)]*x[d,e, k] for d in days for e in employee for k in desks if e in employee_per_desk[k]])

#Constraint 1 = Each employee must stay at the office at least twice a week
for e in employee:
    assignment += lpSum([x[d,e,k] for d in days for k in desks if e in employee_per_desk[k]]) >= 2

#Constraint 2 = Each desk must be occupied by at most one specific employees at each day
for k in desks:
    for d in days:
        assignment += lpSum([x[d,e,k] for e in employee_per_desk[k]]) <= 1

#Constraint 3 - Each employee must be assigned at most once a day
for d in days:
    for e in employee:
        assignment += lpSum([x[d,e,k] for k in desks if e in employee_per_desk[k]]) <= 1

#Constraint 4 - Each desk must be assigned at most once a day
for k in desks:
    for d in days:
        assignment += lpSum([x[d,e,k] for e in employee if e in employee_per_desk[k]]) <= 1


#Constraint 5 = Employees should not be isolated in a zone
for d in days:
    for z in zones:
        for g in groups:
            for m in employee_per_group[g]:
                assignment += lpSum([x[d,m,k] for k in desks_per_employee[m] if k in desks_per_zone[z]]) <= lpSum([x[d,e,k] for e in employee_per_group[g] if e!=m for k in desks_per_employee[e] if k in desks_per_zone[z]])


#Constraint 6 = Each group should have one meeting per week
for g in groups:
    assignment += lpSum([u[d,g] for d in days]) >= 1

#Constraint 7 = All the members of a group should be present once a week
for d in days:
    for g in groups:
        employees_group = employee_per_group[g]
        assignment += lpSum([x[d,e,k] for e in employees_group for k in desks if e in employee_per_desk[k]]) >= len(employees_group)*u[d,g]

In [ ]:
#Solving the model
solve_the_model = assignment.solve(PULP_CBC_CMD(timeLimit=3600, keepFiles=True))
print(f"Status do problema: {LpStatus[solve_the_model]}")

#Print variables
for var in assignment.variables():
    if var.varValue > 0:
        print(f"{var.name}={var.varValue}")

#Print objective function value
print(f"Total Happiness: {value(assignment.objective)}")

Status do problema: Not Solved
u_('J',_'G10')=0.56635405
u_('J',_'G11')=0.41130652
u_('J',_'G2')=0.85714286
u_('J',_'G3')=0.27027092
u_('J',_'G9')=0.49430444
u_('L',_'G0')=0.3478477
u_('L',_'G1')=0.44097452
u_('L',_'G7')=0.59147511
u_('Ma',_'G10')=0.43364595
u_('Ma',_'G4')=0.36714463
u_('Ma',_'G5')=0.39358973
u_('Ma',_'G6')=0.54290514
u_('Mi',_'G0')=0.37062123
u_('Mi',_'G11')=0.32627155
u_('Mi',_'G3')=0.72972908
u_('Mi',_'G4')=0.63285537
u_('Mi',_'G5')=0.60641027
u_('Mi',_'G7')=0.40852489
u_('Mi',_'G9')=0.32763777
u_('V',_'G0')=0.28153107
u_('V',_'G1')=0.55902548
u_('V',_'G11')=0.26242193
u_('V',_'G2')=0.14285714
u_('V',_'G6')=0.45709486
u_('V',_'G8')=1.0
u_('V',_'G9')=0.1780578
x_('J',_'E0',_'D28')=0.6780578
x_('J',_'E0',_'D7')=0.22050601
x_('J',_'E0',_'D8')=0.10143619
x_('J',_'E14',_'D3')=0.3219422
x_('J',_'E14',_'D30')=0.6780578
x_('J',_'E16',_'D0')=0.3219422
x_('J',_'E16',_'D4')=0.6780578
x_('J',_'E17',_'D31')=0.6780578
x_('J',_'E17',_'D35')=0.3219422
x_('J',_'E18',_'D37')=0.321942

In [ ]:
#Print results organized per day
for d in days:
    print(f"-------------------Day: {d}----------------------")
    for k in desks:
        for e in employee:
            if e in employee_per_desk[k]:
                if x[d, e, k].value() == 1:
                    print(f"Desk: {k}, employee:{e}")
    for g in groups:
        if u[d, g].value() == 1:
            group_employees = employee_per_group[g]
            print(f"Meeting of the day: Group {g} - {group_employees}")

-------------------Day: L----------------------
-------------------Day: Ma----------------------
-------------------Day: Mi----------------------
-------------------Day: J----------------------
-------------------Day: V----------------------
Meeting of the day: Group G8 - ['E56', 'E57', 'E58', 'E59', 'E60', 'E61']


In [ ]:
# --- FIM DA MEDIÇÃO ---
end_time = time.time()

# Calcula o tempo total decorrido
time_taken = end_time - start_time

# Imprime o tempo de execução formatado
print(f"\n--- Tempo de Execução do Código ---")
print(f"O processo completo levou {time_taken:.4f} segundos para ser executado.")
print(f"-----------------------------------")


--- Tempo de Execução do Código ---
O processo completo levou 99.9804 segundos para ser executado.
-----------------------------------
